In [1]:
import numpy as np
import pandas as pd

http= "https://data.4tu.nl/file/1c51b037-bf30-4c1a-9c50-25d1617a4899/65710417-8249-40db-a47a-dfa4d25bfad4"
df = pd.read_csv(http, sep=";", parse_dates=['TIMESTAMP'], encoding='cp1252', low_memory=False)
print(df.columns)
df.head()

Index(['CustomerID', 'AgeCategory', 'Gender', 'Office_U', 'Office_W',
       'SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info', 'page_action_detail_EN',
       'service_detail_EN', 'tip_EN'],
      dtype='object')


,CustomerID,AgeCategory,Gender,Office_U,Office_W,SessionID,IPID,TIMESTAMP,VHOST,URL_FILE,PAGE_NAME,REF_URL_category,page_load_error,page_action_detail,tip,service_detail,xps_info,page_action_detail_EN,service_detail_EN,tip_EN
0,2025826,50-65,V,313,313,12956475,620841,2015-10-05 10:12:56.880000000,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/uwv-die...,50plus,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025826,50-65,V,313,313,13243433,620841,2015-09-30 15:14:35.943000000,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/uwv-die...,50plus,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1503890,30-39,V,247,247,14805466,1690840,2015-09-01 19:35:06.707000000,digid.werk.nl,/portal/page/portal/home/diensten/aanvragen-tw,aanvragen-tw,NaN,0,NaN,NaN,3. Versturen gegevens,NaN,NaN,3. Send data,NaN
3,2063574,50-65,M,296,301,12710639,1632512,2015-11-06 10:47:42.137000000,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/uwv-die...,50plus,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2185161,18-29,V,327,327,44281847,757955,2016-01-11 18:44:07.877000000,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/bijstan...,bijstandsuitkering,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], format="mixed")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7174934 entries, 0 to 7174933
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   CustomerID             int64         
 1   AgeCategory            object        
 2   Gender                 object        
 3   Office_U               int64         
 4   Office_W               int64         
 5   SessionID              int64         
 6   IPID                   int64         
 7   TIMESTAMP              datetime64[ns]
 8   VHOST                  object        
 9   URL_FILE               object        
 10  PAGE_NAME              object        
 11  REF_URL_category       object        
 12  page_load_error        int64         
 13  page_action_detail     object        
 14  tip                    object        
 15  service_detail         object        
 16  xps_info               object        
 17  page_action_detail_EN  object        
 18  service_detail_EN     

In [7]:
class Node:
    def __init__(self, name, parent=None):
        self.name = name
        self.children = []
        if parent is not None:
            self.parent_name = parent.name
        else:
            self.parent_name = None

    def add_child(self, child):
        for c in self.children:
            if c.name == child.name:
                return
        self.children.append(child)

In [8]:
import numpy as np
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import product

In [9]:
## =======================================================================
## Trie node class, mostly used to track the path that the process takes
## =======================================================================
class TrieNode:
    def __init__(self, keys, depth=0, parent=None):
        ## Ensure that the keys that we have been given are actually a list
        if type(keys) is list:
            self.keys = keys
        else:
            self.keys = [keys]
        self.parent = parent
        ## Did we get a parent, update our depth then
        if self.parent is not None:
            self.depth = self.parent.depth + 1
        else:
            self.depth = depth

        ## Initial a list of children
        self.children = []
        ## How many times was the noode, or it was a head/tail
        self.count, self.head_count, self.tail_count = 1, 0, 0

    def updateChlidren(self, keys):
        for c in self.children:
            if c.keys == keys:
                return c

        c = TrieNode(keys, parent=self)
        self.children.append(c)
        return c

    def printTree(self, offset=0):
        print("    "*self.depth + f" => ({self.depth}, {offset}) [ keys : {str(self.keys)} ]")
        child_offset = 0
        for c in self.children:
            c.printTree(child_offset)
            child_offset += 1

In [10]:
## =======================================================================
## Trie to build process paths and allow us to calculate statistics using
##     pandas dataframes.
## =======================================================================
class ProcessTrie:
    def makeEmpthDict():
        return {
            "entity": [],
            "start_activity": [],             "stop_activity": [],
            "start_timestamp": [],            "stop_timestamp": [],
            "latitude1": [],                    "longitude1": [],
            "latitude2": [],                    "longitude2": [],
            "resuources": []
        }

    ## Constructor
    def __init__(self, data=None, entity_col="entity", entity_cols=None,
                 activity_col="activity", activity_cols=None,
                 timestamp_col="timestamp", resource_col="resources",
                 latitude_col="latitude", longitude_col="longitude",
                 looping=False, max_step=10000,
                 sep_tails=True, sep_heads=True,
                 debug=False,
                ):
        self.processTrees = []
        self.dataframe = None

        ## What are the definition for our column names
        if type(entity_col) is list: self.entity_cols = entity_col
        elif entity_cols is None:    self.entity_cols = [entity_col]
        else:                        self.entity_cols = entity_cols
        ## Activity Columns
        if type(activity_col) is list: self.activity_cols = activity_col
        elif activity_col is None:     self.activity_cols = [activity_col]
        else:                          self.activity_cols = activity_cols

        self.timestamp_col = timestamp_col
        self.resource_col = resource_col
        self.latitude_col = latitude_col
        self.longitude_col = longitude_col

        ## Some variables for tracking if something is looping and has
        ##     a number of max loops (only way I currently determine the
        ##     end of loops).  Also, should we track the nodes that are
        ##     nodes/heads seperate from the regular middle nodes.
        self.looping, self.max_step, self.sep_tails, self.sep_heads = looping, max_step, sep_tails, sep_heads
        self.debug = debug

        if data is not None:
            self.buildFromDataFrame(data)

    ## Update/Append to the head's of our different process tree's
    def updateHeadTrees(self, keys):
        f = False
        t = None

        key_copy = keys.copy()
        for t in self.processTrees:
            if set(t.keys) == set(key_copy):
                f = True
                break

        if not f:
            t = TrieNode(keys, depth=0)
            self.processTrees.append(t)

        if self.sep_heads == True:
            t.head_count += 1

        return t

    ## Build / expand our process_trie and dataframe from the
    ##     dataframe that we are passed as a parameter
    def buildFromDataFrame(self, data):
        records = ProcessTrie.makeEmpthDict()

        ## Make a copy of our data, and sort the values so that they come up correct
        sort_by = [c for c in self.entity_cols]
        sort_by.append(self.timestamp_col)
        sort_by.extend(self.activity_cols)
        #print(f"-> Sort values by {str(sort_by)}")
        data = data.copy().sort_values(sort_by)

        prev_entity          = [data.iloc[0][c] for c in self.entity_cols]
        prev_activity        = [data.iloc[0][c] for c in self.activity_cols]
        prev_timestamp       = data.iloc[0][self.timestamp_col]
        if self.sep_heads == True: prev_activity.append("HEAD")
        prev_node            = self.updateHeadTrees(prev_activity)
        if self.sep_heads == True: prev_node.head_count += 1
        else:                      prev_node.count += 1

        # What step / edge are we on
        step = 0
        edge = 0
#        print(f"entity: {prev_entity}, \t activity: {prev_activity}, \t timestamp: {prev_timestamp}, \t step: {step}")

        ## Loop through our data-frame and start building nodes
        for index, row in data.iloc[1:].iterrows():
            entity          = [row[c] for c in self.entity_cols]
            activity        = [row[c] for c in self.activity_cols]
            timestamp       = row[self.timestamp_col]
            ## Increment the stamp that we are currently doing
            step += 1
#            print(f"entity1: {entity}, \t activity: {activity}, \t timestamp: {timestamp}, \t step: {step}")

            ## If our previous and current entity are different, clear out
            ##     the previous_node and find the head of the new tree
            if set(prev_entity) != set(entity) or (self.looping and step > self.max_step):
#                print(f"Number of edges counted: {edge}")
#                print(f"Number of edges in array ('stop_activity'): {len(records['stop_activity'])}")

                ## Check if we are tracking tail nodes, if so update our prev_node,
                ##     to increment the number of tails that this node has been
                if self.sep_tails == True:
                    ## Check to see if "Tail is already in our list of activities
                    if "TAIL" not in records["stop_activity"][edge-1]:
                        records["stop_activity"][edge-1].append("TAIL")
                        prev_node.tail_count += 1

                ## Check if we are tracking head nodes sperately, and update
                ##     activities as appropriate
                if self.sep_heads == True: activity.append("HEAD")
                prev_node = self.updateHeadTrees(activity)
                if self.sep_heads == True: prev_node.head_count += 1
                ## Reset the number of steps to 0
                step = 0
#                print(f"entity2: {entity}, \t activity: {activity}, \t timestamp: {timestamp}, \t step: {step}")

            ## Otherwise update our previous_node, base upon our current activity
            else:
                prev_node = prev_node.updateChlidren(activity)
                records["entity"].append(entity)
                records["start_activity"].append(prev_activity)
                records["stop_activity"].append(activity)
                records["start_timestamp"].append(prev_timestamp)
                records["stop_timestamp"].append(timestamp)
                records["latitude1"].append(None)
                records["longitude1"].append(None)
                records["latitude2"].append(None)
                records["longitude2"].append(None)
                records["resources"].append(None)
                ## Increment our edge count
                edge += 1

#                print(f"entity3: {entity}, \t activity: {activity}, \t timestamp: {timestamp}, \t step: {step}")

            ## Set our previous values to the current values
            prev_node.count += 1
            prev_entity = entity
            prev_activity = activity
            prev_timestamp = timestamp

        ## Duplicate copy of records, saved just for trouble shooting
        if self.debug: self.records = records
        ## Update our dataframe
        if self.dataframe is None: self.dataframe = pd.DataFrame(records)
        else:                      self.dataframe = pd.concat([self.dataframe, pd.DataFrame(records)])

In [11]:
df.head()

,CustomerID,AgeCategory,Gender,Office_U,Office_W,SessionID,IPID,TIMESTAMP,VHOST,URL_FILE,PAGE_NAME,REF_URL_category,page_load_error,page_action_detail,tip,service_detail,xps_info,page_action_detail_EN,service_detail_EN,tip_EN
0,2025826,50-65,V,313,313,12956475,620841,2015-10-05 10:12:56.880,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/uwv-die...,50plus,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025826,50-65,V,313,313,13243433,620841,2015-09-30 15:14:35.943,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/uwv-die...,50plus,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1503890,30-39,V,247,247,14805466,1690840,2015-09-01 19:35:06.707,digid.werk.nl,/portal/page/portal/home/diensten/aanvragen-tw,aanvragen-tw,NaN,0,NaN,NaN,3. Versturen gegevens,NaN,NaN,3. Send data,NaN
3,2063574,50-65,M,296,301,12710639,1632512,2015-11-06 10:47:42.137,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/uwv-die...,50plus,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2185161,18-29,V,327,327,44281847,757955,2016-01-11 18:44:07.877,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/bijstan...,bijstandsuitkering,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
class UWV_Columns():
    sessions_id = ["SessionID"]
    activity_id = ["VHOST", "URL_FILE", "PAGE_NAME"]
    timestamp   = "TIMESTAMP"

    def __init__(self):
        pass


## =============================================================================
## Custom UWV node object
## =============================================================================
class UWV_Node():
    def __init__(self, key={"Default"}, depth=0):
        self.key           = key
        self.children      = []
        self.depth         = depth
        self.count         = 1
        self.refresh_count = 0

    ## =========================================================================
    ## Compare the key or node to ourself
    ## =========================================================================
    def compare(self, key=None):
        if key is not None:
            return self.key == key
        else:
            return False

    ## =========================================================================
    ## Update our children, based upon key based
    ## =========================================================================
    def udpateChild(self, key=None):
        self.count += 1

        if self.compare(key=key):
            self.refresh_count += 1
            return self

        for c in self.children:
            if c.compare(key=key):
                return c
        c = UWV_Node(key, depth=self.depth+1)
        self.children.append(c)
        return c


    def __str__(self):
        if self.depth >= 10:
            return f"{'  '*self.depth}({self.depth}) Node: (key: {self.key}, count: {self.count}, refresh_count: {self.refresh_count}) - SKIPPING CHILDREN TO PREVENT RECURSION ERROR"

        to_return = ""
        for c in self.children:
            to_return += f"\n{str(c)}"
        return f"{'  '*self.depth}({self.depth}) Node: (key: {self.key}, count: {self.count}, refresh_count: {self.refresh_count}) {to_return}"

## =============================================================================
## Custom UWV class object, for mapping out the various different session
## =============================================================================
class UWV_ProcessTrie():
    def __init__(self, dataframe=None):
        self.dataframe = dataframe

        ## Get our Column Definitions
        self.cols      = UWV_Columns()
        ## Clean out our nodes and records
        self.nodes     = []
        self.records   = {}
        ## If we got a data-frame, then go ahead and build our data
        if dataframe is not None:
            self.buildFromDataFrame(dataframe)


    ## =========================================================================
    ## Get the set of values associated with the list of columnIds associated
    ## =========================================================================
    def getValue(self, columns=None, row=None):
        if columns is None or row is None:
            return None
        return set([row[c] for c in columns])

    ## =========================================================================
    ## Build our new records/nodes from the data-frame we are getting
    ## =========================================================================
    def updateNodes(self, key):
        for n in self.nodes:
            if n.compare(key=key):
                return n

        n = UWV_Node(key)
        self.nodes.append(n)
        return n

    ## =========================================================================
    ## Build our new records/nodes from the data-frame we are getting
    ## =========================================================================
    def buildFromDataFrame(self, dataframe=None):
        ## If we recieved a data, then update our data-frame and clean
        if dataframe is not None:
            self.dataframe = dataframe

        self.nodes   = []
        self.records = {}

        ## Order by the SessionID and Timestamp
        sort_by = [c for c in self.cols.sessions_id]
        sort_by.append(self.cols.timestamp)
#        dataframe    = self.dataframe.sort_values(["SessionID", "TIMESTAMP"])
        dataframe    = self.dataframe.sort_values(sort_by)
        ## Retrieve the first row
        prev_row     = dataframe.iloc[0]
        prev_id      = self.getValue(self.cols.sessions_id, prev_row)
        activity_id  = self.getValue(self.cols.activity_id, prev_row)
        node         = self.updateNodes(activity_id)


        ## Loop through our data
        for index, row in dataframe.iloc[1:].iterrows():
            ## Look at the prev_row and the new row
            row_id      = self.getValue(self.cols.sessions_id, row)
            activity_id = self.getValue(self.cols.activity_id, row)

            ## Check to see if our two set's match up, if so then update our
            ##     current node / records
            if row_id == prev_id:
                node = node.udpateChild(activity_id)
            else:
                node = self.updateNodes(activity_id)

            ## Update our prev_id and prev_row to the current row / row_id
            prev_id = row_id
            prev_row = row


In [13]:
trie = UWV_ProcessTrie(df[:100000])
for n in trie.nodes:
    print(f"{n}")

(0) Node: (key: {'vacatures_bij_mijn_cv', 'digid.werk.nl', '/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_bij_mijn_cv'}, count: 84090, refresh_count: 84086) 
  (1) Node: (key: {'/werk_nl/werknemer/eintake/inschrijven', 'inschrijven', 'digid.werk.nl'}, count: 1, refresh_count: 0) 
  (1) Node: (key: {'vacatures_bij_mijn_cv', 'www.werk.nl', '/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_bij_mijn_cv'}, count: 3, refresh_count: 0) 
    (2) Node: (key: {'vacatures_bij_mijn_cv', 'digid.werk.nl', '/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_bij_mijn_cv'}, count: 25, refresh_count: 24) 
(0) Node: (key: {'/werk_nl/werknemer/eintake/inschrijven', 'inschrijven', 'digid.werk.nl'}, count: 18, refresh_count: 17) 
(0) Node: (key: {'inschrijven', 'www.werk.nl', '/werk_nl/werknemer/uitkering-aanvragen/inschrijven'}, count: 28, refresh_count: 20) 
  (1) Node: (key: {'vacatures_bij_mijn_cv', 'digid.werk.nl', '/werk_nl/werknemer/mijn_werkmap/werk-zoeken/vacatures_bij_mijn_cv'}, co